# Mock Gameday Run

In [1]:
import pandas as pd
import cvxpy as cp
import numpy as np
from maps import team_map_2
from sim import BacktestPredictionsTable

# Prepare Ref Data & Predictions

In [ ]:
sal = pd.read_csv("ref/DKSalariesExample.csv")
sal['name'] = sal.apply(lambda x: x['Name'].replace(" ", "").upper() if (x['Position'] != 'DST')
                            else team_map_2[x['Name'].replace(" ","")], axis=1)
sal['team1'] = sal['Game Info'].apply(lambda x: x.split(" ")[0].split("@")[0])
sal['team2'] = sal['Game Info'].apply(lambda x: x.split(" ")[0].split("@")[1])
del sal['Game Info']
sal['opp'] = sal.apply(lambda x: x['team2'] if x['team1'] == x['TeamAbbrev'] else x['team1'], axis=1)
sal['date'] = pd.Timestamp("2021.09.01")
sal['year'] = 2021
sal['week'] = 1
btp = BacktestPredictionsTable(seasons=[2018,2019,2020,2021], refresh=False)
btp.build(matchups=sal)

In [ ]:
predictions = btp.table.reset_index().set_index("level_0")[0].rename("pred")
results = sal.join(predictions, on=['name'])

# Solve The Optimization Problem

In [ ]:
num_teams_to_stack = 1
num_players_in_stack = 2
nlineups = 3

pos_stack = results[results['Roster Position'].apply(lambda y: not y in ['QB', 'DST', 'RB'])] \
    .groupby('TeamAbbrev') \
    .apply(lambda grp: grp.sort_values('pred') \
           .tail(num_players_in_stack - 1)) \
    .reset_index(drop=True)
qb_stack = results[results['Roster Position'] == 'QB'].groupby('TeamAbbrev') \
    .apply(lambda grp: grp.sort_values('pred') \
           .tail(1)) \
    .reset_index(drop=True)
stacks = pd.concat([qb_stack, pos_stack]).sort_values('TeamAbbrev')
stacks['DK salary'] = stacks['Salary'].astype(float)
stacked_teams_frame = stacks.groupby('TeamAbbrev').sum().sort_values('pred')
stacked_teams = stacked_teams_frame.index.values[-num_teams_to_stack:]

S = results['Salary'].values
E = results['pred'].values
T = results['TeamAbbrev'].values
P = results['Roster Position'].apply(lambda x: x.replace("/FLEX","")).values
N = results['name'].values
total = {}
for team in stacked_teams:
    prior_lineups = []
    for i in range(int(nlineups / num_teams_to_stack)):
        X = cp.Variable(len(E), boolean=True)
        constraints = [
            sum(X) == 9,
            X @ S <= 50000,
            X @ (P == "QB").astype(float) == 1,
            X @ (P == "TE").astype(float) >= 1,
            X @ (P == "RB").astype(float) >= 2,
            X @ (P == "WR").astype(float) >= 3,
            X @ (P == "DST").astype(float) == 1]
        for n in stacks[stacks.TeamAbbrev == team].name.values:
            constraints.append(X[np.where(N == n)[0][0]] == 1)
        for lineups in prior_lineups:
            constraints.append(X @ lineups <= 8)
        obj = cp.Maximize(X @ E)
        prob = cp.Problem(obj, constraints)
        prob.solve()
        try:
            prior_lineups.append(X.value.copy())
        except AttributeError:
            print("Problem with %s" % team)

In [ ]:
pd.DataFrame([
    N[prior_lineups[0].astype(bool)],
    P[prior_lineups[0].astype(bool)],
    T[prior_lineups[0].astype(bool)],
    S[prior_lineups[0].astype(bool)]]).T[0].values